### Maximum Depth Analysis

In [2]:
import os
from itertools import cycle
from copy import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse, Circle
import matplotlib.gridspec as gridspec
import cufflinks as cf
import plotly.offline

from IPython.display import display, HTML
from ipywidgets import interactive_output, HBox, VBox, Layout 
from ipywidgets import Dropdown, Checkbox, ToggleButtons, SelectionRangeSlider, SelectionSlider
import ipywidgets as widgets

cf.go_offline()
pd.set_option('display.max_columns', 100)

in_dir = '../../output/update/csv/'
main_fp = os.path.join(in_dir, 'results.csv')
retrain_fp = os.path.join(in_dir, 'retrain.csv')

main_df = pd.read_csv(main_fp)
retrain_df = pd.read_csv(retrain_fp)

main_df

,dataset,operation,criterion,n_estimators,max_depth,subsample_size,topd,min_support,model,id,acc_mean,auc_mean,ap_mean,acc_std,auc_std,ap_std,acc_diff_mean,auc_diff_mean,ap_diff_mean,acc_diff_std,auc_diff_std,ap_diff_std,allotted_time,allotted_time_std,n_naive,n_naive_std,n_model,n_model_std,percent_complete,percent_complete_std,n_nodes_avg,n_nodes_avg_std,n_exact_avg,n_exact_avg_std,n_semi_avg,n_semi_avg_std
0,adult,add,gini,10,20,1,0,2,exact,0,0.821578,0.875792,0.675855,0.000132,0.000029,0.000108,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.720083,0.165158,1.0,0.0,229.2,84.283450,0.007039,0.002588,5842.0000,520.062439,2920.5000,260.031219,0.0000,0.000000
1,adult,add,gini,10,20,1,1,2,dart,1,0.821378,0.875786,0.675573,0.000096,0.000028,0.000135,0.000186,0.000011,0.000246,0.000107,0.000033,0.000062,0.855056,0.209259,1.0,0.0,232.6,102.153316,0.007144,0.003137,5746.8000,534.367383,2871.9000,267.183691,1.0000,0.000000
2,adult,add,gini,10,20,1,2,2,dart,2,0.821387,0.875841,0.675684,0.000154,0.000041,0.000185,0.000097,-0.000059,0.000102,0.000132,0.000039,0.000079,0.875003,0.223101,1.0,0.0,312.2,102.563151,0.009588,0.003150,5510.4400,446.941146,2751.8000,223.417155,2.9200,0.083666
3,adult,add,gini,10,20,1,3,2,dart,3,0.821503,0.875855,0.675517,0.000195,0.000049,0.000194,0.000006,-0.000070,0.000218,0.000218,0.000051,0.000115,0.889190,0.207274,1.0,0.0,347.0,86.518784,0.010657,0.002657,5247.6800,483.771983,2617.0400,241.828127,6.3000,0.264575
4,adult,add,gini,10,20,1,4,2,dart,4,0.821414,0.875774,0.675540,0.000146,0.000034,0.000139,0.000109,0.000008,0.000150,0.000195,0.000049,0.000224,0.908687,0.072791,1.0,0.0,435.0,62.529993,0.013360,0.001920,5098.4000,465.179127,2536.9800,232.371474,11.7200,0.460435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,vaccine,delete,gini,250,10,100,6,2,dart,568,0.781109,0.864679,0.846342,0.001262,0.000549,0.000783,0.006029,0.002738,0.002717,0.000781,0.000177,0.000307,18.447209,2.410502,1.0,0.0,820.8,136.799854,0.038418,0.006403,602.7152,6.151122,255.4280,3.017802,45.4296,0.391873
569,vaccine,delete,gini,250,10,100,7,2,dart,569,0.777462,0.862839,0.844326,0.001123,0.000505,0.000651,0.009502,0.004709,0.004945,0.001055,0.000183,0.000250,18.473623,2.541317,1.0,0.0,1027.6,167.449395,0.048097,0.007838,570.9632,9.549359,209.1184,3.947335,75.8632,1.007299
570,vaccine,delete,gini,250,10,100,8,2,dart,570,0.770160,0.858638,0.839226,0.001273,0.000503,0.000690,0.016854,0.008770,0.010051,0.000931,0.000159,0.000224,18.959125,2.297334,1.0,0.0,1544.8,293.027644,0.072305,0.013715,555.8592,5.266691,155.3808,1.823786,122.0488,1.021389
571,vaccine,delete,gini,250,10,100,9,2,dart,571,0.762492,0.853780,0.833068,0.001218,0.000939,0.001254,0.024848,0.013385,0.015865,0.001334,0.000461,0.000525,17.865164,2.566874,1.0,0.0,2581.2,413.730226,0.120814,0.019365,550.9616,5.811372,89.7712,1.400809,185.2096,1.883150


In [11]:
def f(dataset, metric, trees, depths):
    
    lines = ['--', '-.', ':', '-']
    colors = ['r', 'g', 'purple', 'orange', 'blue', 'brown']
    linecolors = [(c, l) for c in colors for l in lines]
    lc_cycler = cycle(linecolors)

    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    axs = axs.flatten()

    ax = axs[0]
    ax.set_title('Efficiency')
    ax.set_xlabel('top d (%)')
    ax.set_ylabel('amortized (s)')

    ax = axs[1]
    ax.set_title('Utility')
    ax.set_xlabel('top d (%)')
    ax.set_ylabel(metric)

    # filter results
    df = main_df[main_df['dataset'] == dataset]
    df = df[df['operation'] == 'delete']
    df = df[df['criterion'] == 'gini']
    df = df[df['subsample_size'] == 1]

    exact_df = df[df['model'] == 'exact']
    dart_df = df[df['model'] == 'dart']

    for depth in df['max_depth'].unique():
        results = []

        # baseline
        exact_depth_df = exact_df[(exact_df['n_estimators'] == trees) & (exact_df['max_depth'] == depth)]
        results.append({'topd': 0,
                        'n_model': exact_depth_df['n_model'].values[0],
                        'n_model_std': exact_depth_df['n_model'].values[0],
                        'metric': exact_depth_df['{}_diff_mean'.format(metric)].values[0],
                        'metric_std': exact_depth_df['{}_diff_std'.format(metric)].values[0]})

        # DART
        dart_depth_df = dart_df[dart_df['max_depth'] == depth]
        print(dart_depth_df)
        for row in dart_depth_df.itertuples(index=False):
            if metric == 'acc':
                metric_mean, metric_std = row.acc_diff_mean, row.acc_std
            elif metric == 'auc':
                metric_mean, metric_std = row.auc_diff_mean, row.auc_std
            else:
                metric_mean, metric_std = row.ap_diff_mean, row.ap_std
            results.append({'topd': row.topd / depth,
                            'n_model': row.n_model,
                            'n_model_std': row.n_model_std,
                            'metric': metric_mean,
                            'metric_std': metric_std})

        df2 = pd.DataFrame(results)
        axs[0].errorbar(df2['topd'], df2['n_model'], yerr=df2['n_model_std'], label='depth={}'.format(depth))
        axs[1].errorbar(df2['topd'], df2['metric'], yerr=df2['metric_std'])

    # plot settings
    title = 'Trees: {}'.format(trees)
    axs[0].legend(bbox_to_anchor=(2.8, 1.01), title=title)
    axs[0].set_yscale('log')

    plt.show()
    
# UI
dataset = Dropdown(description='Dataset', options=['surgical', 'vaccine', 'adult',
                                                   'bank_marketing', 'flight_delays', 'diabetes',
                                                   'olympics', 'census', 'credit_card',
                                                   'synthetic', 'higgs'])
metric = SelectionSlider(description='Metric', options=['auc', 'acc', 'bacc', 'ap'], value='acc')
trees = SelectionSlider(description='No. trees', options=[10, 100, 250], value=250)
depths = SelectionRangeSlider(description='Max depth', options=[1, 3, 5, 10, 20], index=(0, 4))

# create ui
box_1 = VBox([dataset, metric])
box_2 = VBox([trees, depths])
box_3 = VBox([])
ui = HBox([box_1, box_2, box_3])

# reactions
dataset_dict = {'surgical': 'acc', 'adult': 'acc', 'bank_marketing': 'auc',
                'flight_delays': 'auc', 'diabetes': 'acc', 'olympics': 'auc',
                'census': 'auc', 'credit_card': 'ap', 'synthetic': 'acc',
                'higgs': 'acc'}

def update_metric(*args):
    metric.value = dataset_dict[dataset.value]

dataset.observe(update_metric, names='value')

# disply UI
out = interactive_output(f, {'dataset': dataset, 'metric': metric, 'trees': trees, 'depths': depths})
display(ui, out)

Output()